# Preprocess static data from DB Timetables API

- script runs hourly
- to be able to know about every train that is currently on the way, we need data for current_time +- time of the longest trip between stations without stopping, which is 4h (FFM-Berlin Sprinter). Data should last for 1 hour before updating. Therefore we need to request current hour, the 4 hours before that and the 4 hours after that = 9 hours. 
Steps:
- delete data from timetable that is for an hour before current hour - 4. 
- request the timetable from every relevant station, and parse the xmls into a timetable while filtering for Fernverkehr (category in [ICE, IC, EC, NJ ...]). Note which date/hour was used to make the request. 
- artificially add departure and arrival dates where missing
- sort by I. tripid 2. departure time / stop order

### helper functions for handling db dates

In [46]:
import datetime
from dotenv import load_dotenv
from os import getenv
import pandas as pd

load_dotenv()

TIMETABLE_PATH = 'timetable.csv'

# chatgpt generiert lol
import datetime
def dateToDBDate(date: datetime.date) -> str:
    """
    Wandelt ein datetime.date Objekt in einen DB-Date-String "YYMMDD" um.
    """
    return date.strftime("%y%m%d")


def datetimeToDBDatetime(dt: datetime.datetime) -> str:
    """
    Wandelt ein datetime.datetime Objekt in einen DB-Datetime-String "YYMMDDHHMM" um.
    """
    return dt.strftime("%y%m%d%H%M")


def DBDatetimeToDatetime(dbDate: str) -> datetime.datetime:
    """
    Wandelt einen DB-Datetime-String "YYMMDDHHMM" in ein datetime.datetime Objekt um.
    """
    return datetime.datetime.strptime(dbDate, "%y%m%d%H%M")


def DBDateToDate(dbDate: str) -> datetime.date:
    """
    Wandelt einen DB-Date-String "YYMMDD" in ein datetime.date Objekt um.
    """
    return datetime.datetime.strptime(dbDate, "%y%m%d").date()

def datetimeToDBDateAndHourTuple(dt: datetime.datetime):
    date = dt.strftime("%y%m%d")
    hour = dt.strftime("%H")
    return (date, hour)
    
       
print(dateToDBDate(datetime.date(2025, 8, 10)))
print(datetimeToDBDatetime(datetime.datetime(2025, 8, 10, 12, 22)))
print(DBDateToDate("250810"))
print(DBDatetimeToDatetime("2508101222"))
print(datetimeToDBDateAndHourTuple(datetime.datetime(2025, 8, 10, 12, 22)))

250810
2508101222
2025-08-10
2025-08-10 12:22:00
('250810', '12')


In [47]:
# calculate which times need to be requested

date_hour_tuples_to_request = []

current_datetime = datetime.datetime.today()

for hour_offset in range(-4,5):
    datetime_to_request = current_datetime + datetime.timedelta(hours=hour_offset)
    date_hour_tuple = datetimeToDBDateAndHourTuple(datetime_to_request)
    date_hour_tuples_to_request.append(datetimeToDBDateAndHourTuple(datetime_to_request))


print(date_hour_tuples_to_request)



[('250819', '12'), ('250819', '13'), ('250819', '14'), ('250819', '15'), ('250819', '16'), ('250819', '17'), ('250819', '18'), ('250819', '19'), ('250819', '20')]


In [48]:
# load stations that need to be requested
df_stations = pd.read_csv('stations.csv', dtype=str)
print(df_stations)

      station_name station_uic
0   Hamburg-Altona     8002553
1  Hamburg Dammtor     8002548
2      Hamburg Hbf     8002549
3  Hamburg-Harburg     8000147
4         Lüneburg     8000238
5           Uelzen     8000168
6            Celle     8000064
7     Hannover Hbf     8000152


In [49]:
# request and process timetables
import requests
import xmltodict
import time
import os.path as path

# load envs
db_timetables_base_url = getenv("db_timetables_base_url")
db_client_id = getenv("db_client_id")
db_client_secret = getenv("db_client_secret")

df_timetable = None

if path.exists(TIMETABLE_PATH):
    df_timetable = pd.read_csv(TIMETABLE_PATH, dtype=str)

    # delete entries older than 4 hours

    four_hours_ago_tuple = datetimeToDBDateAndHourTuple(datetime.datetime.now() - datetime.timedelta(hours=4))
    timestamp_four_hours_ago = f"{four_hours_ago_tuple[0]}{four_hours_ago_tuple[1]}"

    df_timetable = df_timetable.drop(df_timetable[df_timetable['request_timestamp'] < timestamp_four_hours_ago].index)
    
else:
    df_timetable = pd.DataFrame(columns=['trip_id', 'category', 'number', 'station_name', 'arrival_dbdatetime', 'departure_dbdatetime', 'request_timestamp','request_uic'])

xml_timestamp_tuple = []

request_counter = 0


for index, station_row in df_stations.iterrows():
    station_uic_number = station_row['station_uic']
    station_name = station_row['station_name']
    
    print(f"requesting {station_name}")

    # respect ratelimiting
    if request_counter % 45 == 0 and request_counter > 0:
        print("60s pause for ratelimiting...")
        time.sleep(60)
        
    timetable_rows_for_current_station = []
        
    for date_hour_tuple in date_hour_tuples_to_request:
        date_to_request = date_hour_tuple[0] 
        hour_to_request = date_hour_tuple[1] 
        request_timestamp = f"{date_to_request}{hour_to_request}"
        
        # check if entry already exists
        rows_for_timestamp_and_station = df_timetable[(df_timetable['request_timestamp'] == request_timestamp) & (df_timetable['request_uic'] == station_uic_number)]
        no_rows_for_timestamp_and_station = rows_for_timestamp_and_station.shape[0]
        
        
        if no_rows_for_timestamp_and_station > 0:
            print(f"{date_to_request} / {hour_to_request}:00 / {station_name} already present")
            # already fetched
            continue
        
        # prepare request headers
        auth_headers = {
            'DB-Client-Id': db_client_id,
            'DB-Api-Key': db_client_secret
        }
        
        # {hour:02} means print hour, fill (0) up to 2 (2) digits
        url = f"{db_timetables_base_url}/plan/{station_uic_number}/{date_to_request}/{hour_to_request}"
        
        request_counter += 1
        
        response = requests.get(url=url, headers=auth_headers)
        if response.status_code != 200:
            print(f"skipping hour {hour_to_request}, station {station_uic_number}, {response.status_code}")
            continue
        
        
        # process response
        timetable_dict = xmltodict.parse(response.content)
        timetable_stops = timetable_dict['timetable']['s']
        
        # if there is only one trip in the requested hour, the xml parser parses the timetable stop entry into a dict rather than a list
        if not isinstance(timetable_stops, list):
            timetable_stops = [timetable_stops]
        
        for timetable_stop in timetable_stops:
            try:
                # category (e.g. ICE, RE, S)
                trip_label = timetable_stop['tl']
                category = trip_label['@c']
                number = trip_label['@n']
                
                # for some idiotic reason, the ids can start with a dash while also being separated by a dash
                stop_id = timetable_stop['@id']
                
                has_dash = True if stop_id.startswith("-") else False

                trip_id = None

                if has_dash:
                    # if stop id has dash, the first split result will be empty
                    trip_id = stop_id.split("-")[1]
                    trip_id = f"-{trip_id}"
                else:
                    trip_id = stop_id.split("-")[0]
                
                
                # only save Fernverkehr (not using filter flag because NJ dont have them)
                if category not in ["IC", "EC", "ICE", "FLX", "WB", "RJ", "RJX", "ECE", "EST", "TGV", "NJ", "EN", "ES", "DN", "D", "SJ"]:
                    continue
                
                # arrival
                arrival_dbdatetime = None
                
                if 'ar' in timetable_stop:
                    arrival = timetable_stop['ar']
                    arrival_dbdatetime = arrival['@pt']
                    
                # departure
                departure_dbdatetime = None
                
                if 'dp' in timetable_stop:
                    departure = timetable_stop['dp']
                    departure_dbdatetime = departure['@pt']   
                
                        
                timetable_row = pd.DataFrame(data={'trip_id': [trip_id], 'category':[category], 'number': [number], 'station_name':[station_name], 'arrival_dbdatetime': [arrival_dbdatetime], 'departure_dbdatetime':[departure_dbdatetime], 'request_timestamp': [request_timestamp], 'request_uic': station_uic_number})

                df_timetable = pd.concat([df_timetable, timetable_row], ignore_index=True)
                
            except Exception as e:
                print(e)
    # save after every station 
    df_timetable.to_csv(TIMETABLE_PATH, index=False)

requesting Hamburg-Altona
250819 / 12:00 / Hamburg-Altona already present
250819 / 13:00 / Hamburg-Altona already present
250819 / 14:00 / Hamburg-Altona already present
250819 / 15:00 / Hamburg-Altona already present
250819 / 16:00 / Hamburg-Altona already present
250819 / 17:00 / Hamburg-Altona already present
250819 / 18:00 / Hamburg-Altona already present
250819 / 19:00 / Hamburg-Altona already present
250819 / 20:00 / Hamburg-Altona already present
requesting Hamburg Dammtor
250819 / 12:00 / Hamburg Dammtor already present
250819 / 13:00 / Hamburg Dammtor already present
250819 / 14:00 / Hamburg Dammtor already present
250819 / 15:00 / Hamburg Dammtor already present
250819 / 16:00 / Hamburg Dammtor already present
250819 / 17:00 / Hamburg Dammtor already present
250819 / 18:00 / Hamburg Dammtor already present
250819 / 19:00 / Hamburg Dammtor already present
250819 / 20:00 / Hamburg Dammtor already present
requesting Hamburg Hbf
250819 / 12:00 / Hamburg Hbf already present
250819

In [50]:
# remove duplicates
# duplicates can occur when a train arrives before and departs after the full hour, because in this case the train will be included in both slices

df_timetable = df_timetable.drop_duplicates(subset=['trip_id', 'request_uic', 'departure_dbdatetime'])

In [51]:
# add departure / arrival where necessary

# set arrival to departure where arrival is na
arrival_isna_mask =  df_timetable['arrival_dbdatetime'].isna()
df_timetable.loc[arrival_isna_mask, 'arrival_dbdatetime'] = df_timetable[arrival_isna_mask]['departure_dbdatetime']

# other way around
departure_isna_mask =  df_timetable['departure_dbdatetime'].isna()
df_timetable.loc[departure_isna_mask, 'departure_dbdatetime'] = df_timetable[departure_isna_mask]['arrival_dbdatetime']



In [52]:
# sort dataframe by tripid and departure

df_timetable = df_timetable.sort_values(by=['trip_id','departure_dbdatetime'])

df_timetable.to_csv(TIMETABLE_PATH, index=False)